# Feature Set Generator :: ECDF

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import skew, kurtosis, linregress
from matplotlib import pyplot as plt
import multiprocessing
from multiprocessing import Pool
import os
# import math

In [2]:
# Path settings

# path_var = 1 

# if (path_var == 1):
#     filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'

# else:
#     filepath = '/home/sharan/Desktop/Option_Pricing/datasets/'


filepath = '/home/docboy/Desktop/REL/'


In [3]:

# clean = pd.read_csv(filepath + 'ttm_cleaned_BANKNIFTY_2014_2018_raw.csv')
# clean = clean.dropna()
# filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_BANKNIFTY_2015_2018.csv')
# ts_data = pd.read_csv(filepath + 'INDEXVALUE_BANKNIFTY_2014_2018_combined_raw.csv')


# For NIFTY50

clean = pd.read_csv(filepath + 'ttm_filter_param_0.04_REL_2014_2018.csv')
clean = clean.dropna()
filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_REL_2015_2018.csv')
ts_data = pd.read_csv(filepath + 'INDEXVALUE_REL_2015_2018_combined.csv')


In [4]:
#print(clean)
filt['Date'] = pd.to_datetime(filt['Date'])
filt = filt.sort_values(by='Date')
filt.reset_index(inplace = True, drop = True)
clean['Date'] = pd.to_datetime(clean['Date'])
clean['Expiry'] = pd.to_datetime(clean['Expiry'])
clean = clean.sort_values(by='Date')
clean.reset_index(inplace = True, drop = False)
ts_data['Date'] = pd.to_datetime(ts_data['Date'])  
ts_data = ts_data.sort_values(by='Date')
ts_data.reset_index(inplace = True, drop = False)

# Finding the Scale Parameter

In [5]:
filt['pct'] = filt["Close"]/filt["Strike Price"]*100

In [6]:
# p = round(len(filt)*0.98) 
# q = filt.sort_values(by='pct').reset_index(drop=True)
# scale = q.loc[p, 'pct']/5

# print('scale value is ::', scale)
scale = 1

In [7]:
col = ['Strike Price', 'Open', 'High', 'Low', 'Close', 'Underlying Value']
clean.loc[:, col] = (clean.loc[:, col])**(1/scale)
filt.loc[:, col] = (filt.loc[:, col])**(1/scale)

In [8]:
col = ['Close', 'Open', 'Low', 'High', 'Last Price', 'Prev Close']
ts_data.loc[:, col] = (ts_data.loc[:, col])**(1/scale)

In [9]:
ts_data

,index,Symbol,Date,Prev Close,Last Price,Close,Open,High,Low
0,0,RELIANCE,2015-01-01,891.15,888.00,887.90,886.30,892.00,883.65
1,1,RELIANCE,2015-01-02,887.90,884.95,885.55,888.10,896.05,884.30
2,2,RELIANCE,2015-01-05,885.55,875.55,875.85,885.00,890.90,874.10
3,3,RELIANCE,2015-01-06,875.85,832.50,836.10,870.00,873.00,832.00
4,4,RELIANCE,2015-01-07,836.10,858.00,854.30,837.10,858.80,837.00
5,5,RELIANCE,2015-01-08,854.30,843.00,842.05,860.10,862.90,840.15
6,6,RELIANCE,2015-01-09,842.05,861.50,860.30,848.40,862.00,845.00
7,7,RELIANCE,2015-01-12,860.30,852.50,850.30,860.50,861.00,841.60
8,8,RELIANCE,2015-01-13,850.30,845.00,843.15,853.00,854.95,840.50
9,9,RELIANCE,2015-01-14,843.15,837.00,834.95,843.00,847.00,831.20


In [10]:
clean

,index,Symbol,Date,Expiry,Option Type,Strike Price,Open,High,Low,Close,LTP,Settle Price,No. of contracts,Turnover in Lacs,Open Int,Change in OI,Underlying Value,ttm
0,0,RELIANCE,2014-10-07,2014-10-30,CE,960.0,11.85,13.60,10.50,10.90,10.85,10.90,2075,5041.28,1126250,29000,926.15,23
1,1,RELIANCE,2014-10-07,2014-11-27,CE,940.0,35.00,36.00,32.00,32.75,32.00,32.75,56,136.39,17750,13750,926.15,51
2,2,RELIANCE,2014-10-07,2014-11-27,CE,960.0,29.60,29.60,23.50,24.00,23.50,24.00,175,430.86,60250,41750,926.15,51
3,3,RELIANCE,2014-10-07,2014-10-30,CE,900.0,43.40,46.00,39.10,40.35,39.20,40.35,69,162.53,110500,2750,926.15,23
4,4,RELIANCE,2014-10-07,2014-10-30,CE,920.0,29.75,33.00,26.40,26.95,26.55,26.95,527,1250.39,129000,25750,926.15,23
5,5,RELIANCE,2014-10-07,2014-10-30,CE,940.0,18.35,21.95,16.95,17.60,17.00,17.60,3041,7292.32,825500,207500,926.15,23
6,6,RELIANCE,2014-10-08,2014-10-30,CE,900.0,40.85,51.00,40.85,49.90,49.50,49.90,43,101.77,108500,-2000,936.70,22
7,7,RELIANCE,2014-10-08,2014-11-27,CE,940.0,34.50,40.85,34.50,40.85,40.85,40.85,20,48.94,21250,3500,936.70,50
8,8,RELIANCE,2014-10-08,2014-11-27,CE,960.0,23.00,29.30,23.00,28.15,28.50,28.15,36,88.85,65250,5000,936.70,50
9,9,RELIANCE,2014-10-08,2014-10-30,CE,960.0,10.10,15.25,10.10,14.15,13.55,14.15,2672,6503.74,1200500,74250,936.70,22


# Option Data Functions

In [11]:
def ret(data, date, mode='s'):
    '''
    This function takes in 2 primary params. The function outputs a vector of simple return data
    corressponding to previous 20 days raw ts data.
    The function can also output the log returns on changing the value of the mode param to 'log'.

    ## Modification :: The function can also output just the raw rows if mode param is set to 'e'

    # Demo func
    ret(df, '31-Jan-2019')
    ret(df, '31-Jan-2019', 'log')
    ret(df, '31-Jan-2019', 'e')

    '''
#   print(data.loc[k,:])
    if (mode == 's'): # defualt Func logic
        val = pd.DataFrame()
        val.insert(0, "Return", 0)
        temp=((data['Close']-data['Close'].shift(1))/(data['Close'].shift(1)))*100
        val['Return']=temp[1:]

    if (mode == 'log'): # Func logic if log returns is desired
        val = pd.DataFrame()
        val.insert(0, "Log Return", 0)
        temp = np.log(data['Close'])-np.log(data['Close'].shift(1))
        val['Log Return']=temp[1:]

    if (mode == 'e'): # Func logic to extract the raw data
        k = data.index[data['Date']==date][0]
        val = pd.DataFrame()
        val.insert(0, "Close", 0)
        val.insert(0, "High", 0)
        val.insert(0, "Low", 0)
        val.insert(0, "Date", 0)
        val["Close"] = data.loc[k-19:k, "Close"]
        val['High'] = data.loc[k-19:k, "High"]
        val['Low'] = data.loc[k-19:k, "Low"]
        val['Open'] = data.loc[k-19:k, "Open"]
        val['Date'] = data.loc[k-19:k, "Date"]
    val.reset_index(inplace=True, drop=True)
    return val

### Feature Generator

In [12]:
def gen(ts_data, ttm, atmDate, atmError, strike,spot, opPrice, percentage):
    
    log_return = (ret(ts_data, atmDate, mode = 'log'))
    
    log_return = log_return.loc[:,'Log Return'].tolist()
        
    log_return.sort()
    
    return [atmDate] + log_return + [ttm, atmError, strike,spot, opPrice, percentage]

In [13]:
flist=[]
for i in range(1,20):
    fname='f'+ str(i)
    flist.append(fname)
col_name = ['date'] + flist + ['ttm','atmError','strike','spot','opPrice','percentage']

In [14]:
len(filt)

8138

In [15]:
filt.reset_index(drop=True,inplace=True)
# filt
clean.reset_index(drop=True,inplace=True)

In [16]:
# clean.loc[clean["Strike Price"] == 8600.00]

In [17]:
count = 0
# percentage = []

feat = []
params=[]
for index,row in filt.iterrows():
    ttm = row['ttm']
    strike=row['Strike Price']
    spot=row['Underlying Value']
    atmDate=row['Date']
    expDate=row['Expiry']
    close = row['Close']
    opPrice = close
    opsTS = clean.loc[np.logical_and(clean['Strike Price'] == strike, clean['Expiry'] == expDate)]
    opsTS=opsTS.drop("index",1)
    opsTS.reset_index(inplace=True, drop=True)
#     print(atmDate)
    ind = opsTS.index[opsTS['Date']==atmDate]#Gives Index of that day itself
#     print(ind)
#     print(opsTS)
    ind=ind[0]
    if (ind > 20):
        window = 20
    elif (ind < 1):
        continue        
    else:
        window = ind
        
#     if((spot-strike) >= opPrice):
#         continue
    
    percentage = row['pct']/scale
    
    ts = ret(ts_data, atmDate, mode = 'e')
    
    atmError = ((spot/strike)*100)
    
    params.append((ts,ttm,atmDate,atmError,strike,spot,opPrice,percentage))
#     break
    
pool = Pool()
for res in pool.starmap(gen,params):    
    feat.append(res)
    
pool.close()


In [18]:
results=pd.DataFrame.from_records(feat, columns=col_name)
results

,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f16,f17,f18,f19,ttm,atmError,strike,spot,opPrice,percentage
0,2015-01-01,56.000000,98.655556,900.000000,887.900000,32.000000,3.555556,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,28.000000,100.897727,880.000000,887.900000,29.250000,3.323864,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,28.000000,96.510870,920.000000,887.900000,11.450000,1.244565,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01,28.000000,98.655556,900.000000,887.900000,18.750000,2.083333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01,28.000000,103.244186,860.000000,887.900000,43.000000,5.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-01-02,-0.002650,55.000000,98.394444,900.000000,885.550000,33.150000,3.683333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-01-02,-0.002650,27.000000,100.630682,880.000000,885.550000,29.050000,3.301136,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-01-02,-0.002650,27.000000,102.970930,860.000000,885.550000,42.050000,4.889535,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-01-02,-0.002650,27.000000,96.255435,920.000000,885.550000,11.250000,1.222826,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-01-02,-0.002650,27.000000,98.394444,900.000000,885.550000,18.750000,2.083333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
results = results.dropna()

In [20]:
print(len(results))

7416


In [21]:
# results.to_csv(filepath + 'BANKNIFTY_feature_set_ECDF_STEP_1.csv', index = False)

filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'

results.to_csv(filepath + 'REL_feature_set_ECDF_STEP_1_NOSCALED.csv', index = False)



